# Google - Isolated Sign Language Recognition

In [4]:
# Machine Learning and Data Science Imports (basics)
import tensorflow as tf
import random
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
from matplotlib import animation, rc; rc('animation', html='jshtml')
import json
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import plotly.io as pio
pio.templates.default = "simple_white"
import math
import warnings
warnings.filterwarnings("ignore")

In [5]:
# import data from specified data directory
def load_json_file(json_path):
    with open(json_path, 'r') as f:
        sign_map = json.load(f)
    return sign_map

class CFG:
    data_dir = "/kaggle/input/asl-signs/"
    sequence_length = 12
    rows_per_frame = 543
    batch_size = 512

ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)
    
sign_map = load_json_file(CFG.data_dir + 'sign_to_prediction_index_map.json')
train_data = pd.read_csv(CFG.data_dir + 'train.csv')

s2p_map = {k.lower():v for k,v in load_json_file(CFG.data_dir + "sign_to_prediction_index_map.json").items()}
p2s_map = {v:k for k,v in load_json_file(CFG.data_dir + "sign_to_prediction_index_map.json").items()}
encoder = lambda x: s2p_map.get(x.lower())
decoder = lambda x: p2s_map.get(x)

# inspect data from one parquet file
train_data.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [6]:
train_data.shape

(94477, 4)

## Model Pipeline

In [7]:
from sklearn.preprocessing import normalize
train_x = np.load("/kaggle/input/gislr-feature-data/feature_data.npy").astype(np.float32)
train_y = np.load("/kaggle/input/gislr-feature-data/feature_labels.npy").astype(np.uint8)

N_TOTAL = train_x.shape[0]
VAL_PCT = 0.1
N_VAL   = int(N_TOTAL*VAL_PCT)
N_TRAIN = N_TOTAL-N_VAL

random_idxs = random.sample(range(N_TOTAL), N_TOTAL)
train_idxs, val_idxs = np.array(random_idxs[:N_TRAIN]), np.array(random_idxs[N_TRAIN:])

val_x, val_y = train_x[val_idxs], train_y[val_idxs]
train_x, train_y = train_x[train_idxs], train_y[train_idxs]

In [8]:
class GatherLayer(tf.keras.layers.Layer):
    def __init__(self, indices, **kwargs):
        super(GatherLayer, self).__init__(**kwargs)
        self.indices = indices
        
    def call(self, inputs):
        return tf.gather(inputs, tf.cast(self.indices, dtype=tf.int32), axis=1)

class ChooseHand(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def call(self, inputs):
        hand_inputs_final = []
        for s in range(inputs.shape[0]):
            left_hand_inputs = inputs[s, 1404:1467]
            right_hand_inputs = inputs[s, 1566:1629]
            hand_inputs = tf.cond(tf.equal(tf.reduce_sum(tf.abs(left_hand_inputs)), 0),
                              lambda: right_hand_inputs,
                              lambda: left_hand_inputs)
            hand_inputs_final.append(hand_inputs)
        stacked_hand_inputs = tf.stack(hand_inputs_final, axis=-1)
        transposed_hand_inputs = tf.transpose(stacked_hand_inputs, perm=[1, 0])
        return transposed_hand_inputs
    
class HandKineticLayer(tf.keras.layers.Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, tensor):
        end_ldmrk = [4*3,8*3,12*3,16*3, 19*3]
        for index in range(0, 60, 3):
            start = (index+3) if index in end_ldmrk else (index+6)
        
            dist_list = []
            for x in range(start, 63, 3):
                dx = tensor[:,index] - tensor[:,x]
                dy = tensor[:,index+1] - tensor[:,x+1]
                dist = tf.sqrt(tf.square(dx) + tf.square(dy))
                dist_list.append(dist)
                
            dist_tensor = tf.stack(dist_list, axis=-1)
            tensor = tf.concat([tensor, dist_tensor], axis=-1)
        return tensor
    
    
class FaceKineticLayer(tf.keras.layers.Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, tensor):
        for index in range(0, 36, 2):
            start = (index+4)
            
            dist_list = []
            for x in range(start, 40, 2):
                dx = tensor[:,index] - tensor[:,x]
                dy = tensor[:,index+1] - tensor[:,x+1]
                dist = tf.sqrt(tf.square(dx) + tf.square(dy))
                dist_list.append(dist)
            
            dist_tensor = tf.stack(dist_list, axis=-1)
            tensor = tf.concat([tensor, dist_tensor], axis=-1)
        return tensor

    
class AngularLayer(tf.keras.layers.Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, tensor):
        for index in range(0, 60, 3):
            start = (index+3) 
            angle_list = []
            for j in range(start, 63, 3):
                vector = tensor[:, j:j+2] - tensor[:, index:index+2] 
                norms = tf.norm(vector, axis=1)
                x_tilt = (tf.clip_by_value(vector[:, 0]/norms, -1.0, 1.0))
                y_tilt = (tf.clip_by_value(vector[:, 1]/norms, -1.0, 1.0))
                angle = tf.stack([x_tilt, y_tilt], axis=1)
                angle_list.append(angle)
            angle_tensor = tf.concat(angle_list, axis=-1)
            tensor = tf.concat([tensor, angle_tensor], axis=-1)
            tensor = tf.cast(tensor, tf.float32)
        return tensor
    
class FaceHandDistance(tf.keras.layers.Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, tensor):
        start = 657
        for index in range(0, 42, 2):
            dist_list = []
            for x in range(start, 697, 2):
                dx = tensor[:,index] - tensor[:,x]
                dy = tensor[:,index+1] - tensor[:,x+1]
                dist = tf.sqrt(tf.square(dx) + tf.square(dy))
                dist_list.append(dist)
                
            dist_tensor = tf.stack(dist_list, axis=-1)
            tensor = tf.concat([tensor, dist_tensor], axis=-1)
        return tensor

In [9]:
output_bias = tf.keras.initializers.Constant(1.0 / 250.0)
def ResidualBlock(inputs, nn, dropout):
    x = tf.keras.layers.Dense(nn)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("gelu")(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    return x

def GRU(inputs, nn, dropout):
    x = tf.keras.layers.GRU(nn, dropout=0.0, return_sequences=True)(inputs)
    x = tf.keras.layers.GRU(nn, dropout=dropout, return_sequences=False)(inputs)
    return x

class MSD(tf.keras.layers.Layer):
    def __init__(self,units,**kwargs,):
        super().__init__(**kwargs)

        self.lin = tf.keras.layers.Dense(
            units,
            activation=None,
            use_bias=True,
            bias_initializer=output_bias,
        )

        self.dropouts = [tf.keras.layers.Dropout((0.5 - 0.2), seed=135 ),
                tf.keras.layers.Dropout((0.5 - 0.1), seed=690 ),
                tf.keras.layers.Dropout((0.5), seed=275 ),
                tf.keras.layers.Dropout((0.5 + 0.1), seed=348 ),
                tf.keras.layers.Dropout((0.5 + 0.2), seed=861),]

    def call(self, inputs):
        for ii, drop in enumerate(self.dropouts):
            if ii == 0:
                out = self.lin(drop(inputs)) / 5.0
            else:
                out += self.lin(drop(inputs)) / 5.0
        return out

In [10]:
def build_model():
    inputs = tf.keras.Input(batch_shape=(CFG.batch_size,) + (3258,), dtype=tf.float32)
    hand_inputs = ChooseHand()(inputs) 
    hand_inputs = HandKineticLayer()(hand_inputs)  
    hand_inputs = AngularLayer()(hand_inputs)
    hand_coord = hand_inputs[:, :42]
    hand_features = hand_inputs[:, 63:]
    hand_inputs = tf.keras.layers.Concatenate(axis=1)([hand_coord, hand_features])
    
    lower_lip_indices = [78*3, 191*3, 80*3, 81*3, 82*3, 13*3, 312*3, 311*3, 310*3, 415*3,
            95*3, 88*3, 178*3, 87*3, 14*3, 317*3, 402*3, 318*3, 324*3, 308*3,
            78*3+1, 191*3+1, 80*3+1, 81*3+1, 82*3+1, 13*3+1, 312*3+1, 311*3+1, 310*3+1, 415*3+1,
            95*3+1, 88*3+1, 178*3+1, 87*3+1, 14*3+1, 317*3+1, 402*3+1, 318*3+1, 324*3+1, 308*3+1]
    
    upper_lip_indices = [61*3, 185*3, 40*3, 39*3, 37*3, 0, 267*3, 269*3, 270*3, 409*3,
            291*3, 146*3, 91*3, 181*3, 84*3, 17*3, 314*3, 405*3, 321*3, 375*3,
            61*3+1, 185*3+1, 40*3+1, 39*3+1, 37*3+1, 0+1, 267*3+1, 269*3+1, 270*3+1, 409*3+1,
            291*3+1, 146*3+1, 91*3+1, 181*3+1, 84*3+1, 17*3+1, 314*3+1, 405*3+1, 321*3+1, 375*3+1]
    
    lower_lip_inputs = GatherLayer(lower_lip_indices)(inputs)
    lower_lip_inputs = FaceKineticLayer()(lower_lip_inputs)
    
    upper_lip_inputs = GatherLayer(upper_lip_indices)(inputs)
    upper_lip_inputs = FaceKineticLayer()(upper_lip_inputs)
    
    upper_body_indices = [16*3, 14*3, 12*3, 11*3, 13*3, 15*3,
                          16*3+1, 14*3+1, 12*3+1, 11*3+1, 13*3+1, 15*3+1]
    pose_inputs = GatherLayer(upper_body_indices)(inputs)
    
    all_inputs = tf.keras.layers.Concatenate(axis=1)([hand_inputs, lower_lip_inputs, upper_lip_inputs, pose_inputs])
#     all_inputs = FaceHandDistance()(all_inputs)
    
    all_conv = tf.reshape(all_inputs, (-1, 1, all_inputs.shape[1]))
    
    vector = GRU(all_conv, 1024//2, 0.5)     
    vector = ResidualBlock(vector, 1024//1, 0.25)     
    vector = ResidualBlock(vector, 1024//1, 0.0)
    vector = MSD(1024//4)(vector)
    
    vector = tf.keras.layers.Flatten()(vector)
    output = tf.keras.layers.Dense(250, activation="softmax")(vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(tf.keras.optimizers.Adam(0.000333), "sparse_categorical_crossentropy", 
                  metrics =["sparse_top_k_categorical_accuracy", "acc"])
    return model

## Model 1

In [11]:
import keras.backend as K

K.clear_session()
model1 = build_model()
model1.summary()
# tf.keras.utils.plot_model(model1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(512, 3258)]        0           []                               
                                                                                                  
 choose_hand (ChooseHand)       (512, 63)            0           ['input_1[0][0]']                
                                                                                                  
 hand_kinetic_layer (HandKineti  (512, 258)          0           ['choose_hand[0][0]']            
 cLayer)                                                                                          
                                                                                                  
 angular_layer (AngularLayer)   (512, 678)           0           ['hand_kinetic_layer[0][0]'] 

## Train Model

In [12]:
file_name = "model1"
callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            file_name, 
            save_best_only=True, 
            restore_best_weights=True, 
            monitor="val_accuracy",
            mode="max"
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            mode="max",
            patience=5
        ),
        tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.8, verbose=1)
    ]

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_dataset = train_dataset.batch(batch_size=CFG.batch_size, drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(batch_size=CFG.batch_size, drop_remainder=True)

In [18]:

model1.fit(train_dataset, epochs=100, validation_data=val_dataset, batch_size=CFG.batch_size, callbacks=callbacks)

Epoch 1/100
166/166 [==============================] - 114s 324ms/step - loss: 4.4769 - sparse_top_k_categorical_accuracy: 0.2593 - acc: 0.0902 - val_loss: 3.8043 - val_sparse_top_k_categorical_accuracy: 0.4800 - val_acc: 0.2041 - lr: 3.3300e-04
Epoch 2/100
166/166 [==============================] - 22s 131ms/step - loss: 3.5319 - sparse_top_k_categorical_accuracy: 0.4736 - acc: 0.1974 - val_loss: 3.0336 - val_sparse_top_k_categorical_accuracy: 0.5907 - val_acc: 0.2847 - lr: 3.3300e-04
Epoch 3/100
166/166 [==============================] - 22s 133ms/step - loss: 3.2167 - sparse_top_k_categorical_accuracy: 0.5482 - acc: 0.2475 - val_loss: 2.7781 - val_sparse_top_k_categorical_accuracy: 0.6476 - val_acc: 0.3365 - lr: 3.3300e-04
Epoch 4/100
166/166 [==============================] - 22s 131ms/step - loss: 3.0347 - sparse_top_k_categorical_accuracy: 0.5898 - acc: 0.2808 - val_loss: 2.6287 - val_sparse_top_k_categorical_accuracy: 0.6759 - val_acc: 0.3681 - lr: 3.3300e-04
Epoch 5/100
166/166

In [19]:
model1.save('/kaggle/working/Gru_model.h5')

In [26]:
loaded_model = tf.keras.models.load_model(
    "/kaggle/working/Gru_model.h5",
    custom_objects={'ChooseHand': ChooseHand, 'HandKineticLayer': HandKineticLayer, 'AngularLayer': AngularLayer, 'GatherLayer': GatherLayer, 'FaceKineticLayer' : FaceKineticLayer, 'MSD' : MSD}
)

In [30]:
loaded_model.predict(train_data[0])

KeyError: 0